# Начало работы

Для начала запустим ноутбук `prepossess.ipnyb`.
Из него мы получим данные по найденным совпадениям.

Теперь предлагается смонтировать диск (проверялось только в коллабе, не факт, что будет работать в локале)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Теперь надо пофильтровать полученный файл

In [3]:
import pandas as pd
import io

file_path = '/content/drive/MyDrive/colab_data/output.domtblout'

def process_line(line):
    values = line.split()[:7]
    line = ' '.join(values)
    return line

with open(file_path) as f:
    lines = [process_line(line) for line in f if not line.startswith('#')]

column_names = ['target_name', 'accession', 'tlen', 'query_name', 'accession2', 'qlen', 'E-value']
df = pd.read_csv(io.StringIO('\n'.join(lines)), sep=' ', header=None, names=column_names)
df = df[df['E-value'] < 1e-5].reset_index()


Загрузим таблицу в которой будем искать белки, отвечающие за эпигенетику:

In [4]:
ref_df = pd.read_excel('/content/drive/MyDrive/colab_data/For project.xlsx', sheet_name=1)

Найдем совпадения:

In [5]:
from tqdm import tqdm

indexes = [False for i in range(df.shape[0])]
for i, row in tqdm(df.iterrows()):
    query_name = row['query_name']
    accession = row['accession2'].split('.')[0]
    if i > 15000:
        break
    for _, ref_row in ref_df.iterrows():
        domain_description = ref_row['Pfam domains']
        if query_name in domain_description and accession in domain_description:
            indexes[i] = True
            break


15001it [07:11, 34.79it/s]


Подготовим таблицу содержащую 10 генов для уникальных семейств:

In [6]:
ids = {i + ' ' + j: index for index,  i, j in zip(df.index[indexes], df[indexes]['accession2'], df[indexes]['query_name'])}
final_df = df.loc[list(ids.values())[:10], :]
final_df

,index,target_name,accession,tlen,query_name,accession2,qlen,E-value
24,28,CAE7202474.1,-,158,14-3-3,PF00244.23,222,8.900000e-09
32,50,CAE7429361.1,-,269,2-Hacid_dh,PF00389.33,134,5.500000e-15
58,79,CAE7339232.1,-,1010,2-Hacid_dh_C,PF02826.22,178,1.900000e-06
110,244,CAE7605289.1,-,518,2OG-FeII_Oxy_2,PF13532.9,194,1.400000e-06
472,1108,CAE7420740.1,-,322,AAA,PF00004.32,132,6.600000e-06
1327,4208,CAE7451468.1,-,478,AAA_22,PF13401.9,137,9.700000e-06
3274,10823,CAE7206723.1,-,1066,Acetyltransf_1,PF00583.28,117,9.000000e-06
3337,11028,CAE7559748.1,-,1033,Acetyltransf_7,PF13508.10,80,7.200000e-06
3429,11314,CAE7274853.1,-,262,Actin,PF00022.22,385,3.100000e-19
4549,13861,CAE7397939.1,-,454,Amino_oxidase,PF01593.27,452,1.200000e-06


Сейчас нам надо найти скаффолды, которые отвечают за выбранные нами гены

In [7]:
scaffolds = []
for target_name in final_df['target_name']:
    res = !grep $target_name /content/drive/MyDrive/colab_data/ncbi_dataset/data/GCA_905221605.1/genomic.gtf -m 1
    scaffold = res[0].split('\t')[0]
    scaffolds.append(scaffold)

genome_file = open('/content/drive/MyDrive/colab_data/ncbi_dataset/data/GCA_905221605.1/GCA_905221605.1_Snat_CCMP2548_genomic.fna', 'r')
output_file = open('/content/drive/MyDrive/colab_data/cutted_genome.fna', 'w')
genome_lines = genome_file.readlines()

flag = False
for line in genome_lines:
    if line[0] == '>':
        scaffold = line.split()[0][1:]
        if scaffold in scaffolds:
            flag = True
        else:
            flag = False
    if flag:
        output_file.write(line.upper())

genome_file.close()
output_file.close()

Теперь надо обработать выбранные нами участки в `zdna.ipynb` и `zhunt.ipynb`. Эти ноутбуки можно запускать параллельно, нам понадобятся  файлы, которые они соберут.

# Обработка данных

Итак, у нас есть все основные данные, теперь мы может из обработать и проанализировать.

In [9]:
import pandas as pd

gtf_file = '/content/drive/MyDrive/colab_data/ncbi_dataset/data/GCA_905221605.1/genomic.gtf'
gtf_data = pd.read_csv(gtf_file, sep='\t', comment='#', header=None)
gtf_data.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']

Для начала заполним все необходимые структуры

In [10]:
from collections import defaultdict

exons = defaultdict(list)
introns = defaultdict(list)
promoters = defaultdict(list)
downstream = defaultdict(list)
intergenic = defaultdict(list)
gens = defaultdict(list)

for _, row in gtf_data.iterrows():
    scaffold = row['seqname']
    if scaffold not in scaffolds:
        continue

    if row['feature'] == 'exon':
        exons[scaffold].append((row['start'], row['end']))
    elif row['feature'] == 'gene':
        gens[scaffold].append((row['start'], row['end']))
        if row['strand'] == '+':
            promoter_start = max(1, row['start'] - 1000)
            promoter_end = row['start'] - 1
            downstream_start = row['end'] + 1
            downstream_end = row['end'] + 200
        else:
            promoter_start = row['end'] + 1
            promoter_end = row['end'] + 1000
            downstream_start = max(1, row['start'] - 200)
            downstream_end = row['start'] - 1
        promoters[scaffold].append((promoter_start, promoter_end))
        downstream[scaffold].append((downstream_start, downstream_end))

In [11]:
genome_by_scaffolds = {}
with open('/content/drive/MyDrive/colab_data/cutted_genome.fna') as f:
    scaffold = None
    for line in f.readlines():
        if '>' == line[0]:
            scaffold = line[1:].split()[0]
            genome_by_scaffolds[scaffold] = ''
        else:
            assert line[-1] == '\n'
            genome_by_scaffolds[scaffold] += line[:-1]

In [12]:
introns = defaultdict(list)
for scaffold in scaffolds:
    exon_row = exons[scaffold]
    exon_row.sort()
    prev = 0
    for exon in exon_row:
        if prev < exon[0] - 1:
            introns[scaffold].append((prev, exon[0] - 1))
        prev = exon[1] + 1
    next_ind = len(genome_by_scaffolds[scaffold])
    if prev < next_ind:
        introns[scaffold].append((prev, next_ind))

In [13]:
intergenic = defaultdict(list)
for scaffold in scaffolds:
    gen_row = gens[scaffold]
    gen_row.sort()
    prev = 0
    for gen in gen_row:
        if prev < gen[0] - 1:
            intergenic[scaffold].append((prev, gen[0] - 1))
        prev = gen[1] + 1
    next_ind = len(genome_by_scaffolds[scaffold])
    if prev < next_ind:
        intergenic[scaffold].append((prev, next_ind))

In [14]:
import re
from collections import defaultdict

patterns = ["(?:G{3,5}[ATGC]{1,7}){3,}G{3,5}", "(?:C{3,5}[ATGC]{1,7}){3,}C{3,5}"]

quad_coordinates = defaultdict(list)
for pattern in patterns:
    for scaffold in scaffolds:
        for m in re.finditer(pattern, genome_by_scaffolds[scaffold], re.IGNORECASE):
            quad_coordinates[scaffold].append([m.start(), m.end()])

In [15]:
zhunt_coordinates = defaultdict(list)

with open('/content/drive/MyDrive/colab_data/cutted_genome.fna.Z-SCORE', 'r') as f:
    for line in f.readlines()[1:]:
        start, end, _, _, _, score, _, _ = line.split()
        start, end, score = int(start), int(end), float(score)
        if score < 400:
            continue
        for scaffold in scaffolds:
            if start < len(genome_by_scaffolds[scaffold]):
                zhunt_coordinates[scaffold].append([start, end])
                break
            start -= len(genome_by_scaffolds[scaffold])
            end -= len(genome_by_scaffolds[scaffold])

In [16]:
bert_coordinates = defaultdict(list)

with open('/content/drive/MyDrive/colab_data/text_predictions.txt', 'r') as f:
    for row in f.readlines()[1:]:
        if 'CAUY' in row:
            scaffold = row.split('\n')[0]
        else:
            try:
                start, end = map(int, row.split())
                bert_coordinates[scaffold].append([start, end])
            except Exception:
                continue

In [17]:
def get_stat(objects, exp_coordinates):
    all_intersections = set()
    total_count = 0

    for scaffold in scaffolds:
        total_count += len(exp_coordinates[scaffold])
        for start, end in objects[scaffold]:
            for exp_start, exp_end in exp_coordinates[scaffold]:
                if max(start, exp_start) <= min(end, exp_end):
                    all_intersections.add((scaffold, exp_start, exp_end))
    return len(all_intersections), len(all_intersections) / total_count

In [18]:
get_stat(exons, bert_coordinates)

(554, 0.12360553324408746)

In [20]:
get_stat(exons, zhunt_coordinates)

(5731, 0.0838822048534879)

Собираем таблицу для отчёта

In [21]:
data = defaultdict(dict)

for region_name, region_obj in zip(['Exons', 'Introns', 'Promoters (1000 up from TSS)', 'Downstream (200 bp)', 'Intergenic'],
                                   [exons, introns, promoters, downstream, intergenic]):
    for coordinates_name, coordinates in zip(
        ['квадруплексов', 'предсказаний Zhunt', 'предсказаний ZDNABERT'],
        [quad_coordinates, zhunt_coordinates, bert_coordinates]
    ):
        count, perc = get_stat(region_obj, coordinates)
        data[region_name]['Число ' + coordinates_name] = count
        data[region_name]['Доля ' + coordinates_name] = perc


df = pd.DataFrame(data).T
df

,Число квадруплексов,Доля квадруплексов,Число предсказаний Zhunt,Доля предсказаний Zhunt,Число предсказаний ZDNABERT,Доля предсказаний ZDNABERT
Exons,43.0,0.003150,5731.0,0.083882,554.0,0.123606
Introns,13631.0,0.998535,64621.0,0.945830,2556.0,0.570281
Promoters (1000 up from TSS),409.0,0.029961,3474.0,0.050847,239.0,0.053324
Downstream (200 bp),60.0,0.004395,879.0,0.012866,47.0,0.010486
Intergenic,9005.0,0.659659,43000.0,0.629373,1371.0,0.305890


Собираем файлы, которые будут нужны для групповой части

In [22]:
epigenic = open("/content/drive/MyDrive/colab_data/epigenic.fa", 'w')

organism_name = "Symbiodinium natans"

for target, accession, query in zip(final_df['target_name'][:5], final_df['accession2'][:5], final_df['query_name'][:5]):
    res = !grep $target /content/drive/MyDrive/colab_data/ncbi_dataset/data/GCA_905221605.1/genomic.gtf -m 1
    res_parsed = res[0].split()
    gene, l, r = res_parsed[0], res_parsed[3], res_parsed[4]
    epigenic.write(f">{organism_name}__{accession}___{query}\n")
    epigenic.write(f"{genome_by_scaffolds[gene][int(l):int(r)]}\n")
epigenic.close()


In [23]:
promoters_plus = defaultdict(list)
downstream_plus = defaultdict(list)

for _, row in gtf_data.iterrows():
    scaffold = row['seqname']
    if scaffold not in scaffolds:
        continue

    if row['feature'] == 'exon':
        exons[scaffold].append((row['start'], row['end']))
    elif row['feature'] == 'gene':
        gens[scaffold].append((row['start'], row['end']))
        if row['strand'] == '+':
            promoter_start = max(1, row['start'] - 1000)
            promoter_end = row['start'] - 1
            downstream_start = row['end'] + 1
            downstream_end = row['end'] + 200
            promoters_plus[scaffold].append((promoter_start, promoter_end))
            downstream_plus[scaffold].append((downstream_start, downstream_end))

patterns = ["(?:G{3,5}[ATGC]{1,7}){3,}G{3,5}"]

quad_coordinates_plus = defaultdict(list)
for pattern in patterns:
    for scaffold in scaffolds:
        for m in re.finditer(pattern, genome_by_scaffolds[scaffold], re.IGNORECASE):
            quad_coordinates_plus[scaffold].append((m.start(), m.end()))

out = open('/content/drive/MyDrive/colab_data/quad.fa', 'w')
count = 0
for scaffold in scaffolds:
    if count >= 5:
        break
    for start, end in promoters_plus[scaffold]:
        for exp_start, exp_end in quad_coordinates_plus[scaffold]:
            if max(start, exp_start) <= min(end, exp_end):
                out.write(f">{organism_name}__{scaffold}_{count}\n")
                out.write(f"{genome_by_scaffolds[scaffold][exp_start : exp_end + 1]}\n")
                count += 1
                break
out.close()

In [24]:

out = open('/content/drive/MyDrive/colab_data/zdna.fa', 'w')
count = 0
for scaffold in scaffolds:
    if count >= 5:
        break
    for start, end in promoters_plus[scaffold]:
        for exp_start, exp_end in bert_coordinates[scaffold]:
            if max(start, exp_start) <= min(end, exp_end):
                out.write(f">{organism_name}__{scaffold}_{count}\n")
                out.write(f"{genome_by_scaffolds[scaffold][exp_start : exp_end + 1]}\n")
                count += 1
                break
out.close()